In [2]:
from datasets import load_dataset

In [13]:
# both only have train splits
ds_pos_full = load_dataset("Lichess/chess-position-evaluations", streaming=True)
ds_games_full = load_dataset("Lichess/standard-chess-games", streaming=True)

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/25812 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/25812 [00:00<?, ?it/s]

In [14]:
ds_pos = ds_pos_full["train"].take(1000)
ds_games = ds_games_full["train"].take(1000)

In [39]:
print(ds_pos)
for i, example in enumerate(ds_pos):
    print(example)
    print("fen:", example['fen'])
    print("line:", example['line'])
    print("centi-pawn:", example['cp'])
    if i >= 2:  # Limit to 10 examples
        break

IterableDataset({
    features: ['fen', 'line', 'depth', 'knodes', 'cp', 'mate'],
    num_shards: 17
})
{'fen': '7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -', 'line': 'f7g7 e6e2 h8d8 e2d2 b7b5 c4b3 g7f6 d1e1 a6a5 a2a3', 'depth': 46, 'knodes': 4189972, 'cp': 69, 'mate': None}
fen: 7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -
line: f7g7 e6e2 h8d8 e2d2 b7b5 c4b3 g7f6 d1e1 a6a5 a2a3
centi-pawn: 69
{'fen': '7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -', 'line': 'h8d8 d1e1 a6a5 a2a3 c6d7 e6e7 f7f6 e1f2 b7b5 c4b3', 'depth': 46, 'knodes': 4189972, 'cp': 163, 'mate': None}
fen: 7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -
line: h8d8 d1e1 a6a5 a2a3 c6d7 e6e7 f7f6 e1f2 b7b5 c4b3
centi-pawn: 163
{'fen': '7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -', 'line': 'h8a8 d1e1 a6a5 e6h6 f7g7 h6h4 a8d8 c4d3 c6g2 d3f5', 'depth': 46, 'knodes': 4189972, 'cp': 229, 'mate': None}
fen: 7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -
line: h8a8 d1e1 a6a5 e6h6 f7g7 h6h4 a8d8 c4d3 c6g2 d3f5
centi-pawn: 229


In [40]:
print(ds_games)
"""
About 6% of the games include Stockfish analysis evaluations: 
[%eval 2.35] (235 centipawn advantage), 
[%eval #-4] (getting mated in 4), 
always from White's point of view.
"""
found = 0
for i, example in enumerate(ds_games):
    # print(example)
    if 'eval' in example['movetext']:
        print(example['movetext'])
        found += 1
    if found > 4:  # Limit to 10 examples
        break

IterableDataset({
    features: ['Event', 'Site', 'White', 'Black', 'Result', 'WhiteTitle', 'BlackTitle', 'WhiteElo', 'BlackElo', 'WhiteRatingDiff', 'BlackRatingDiff', 'UTCDate', 'UTCTime', 'ECO', 'Opening', 'Termination', 'TimeControl', 'movetext'],
    num_shards: 25812
})
1. e4 { [%eval 0.2] } 1... e6 { [%eval 0.13] } 2. Bc4 { [%eval -0.31] } 2... d5 { [%eval -0.28] } 3. exd5 { [%eval -0.37] } 3... exd5 { [%eval -0.31] } 4. Bb3 { [%eval -0.33] } 4... Nf6 { [%eval -0.35] } 5. d4 { [%eval -0.34] } 5... Be7 { [%eval 0.0] } 6. Nf3 { [%eval 0.0] } 6... O-O { [%eval -0.08] } 7. Bg5 { [%eval -0.19] } 7... h6 { [%eval -0.29] } 8. Bxf6 { [%eval -0.36] } 8... Bxf6 { [%eval -0.37] } 9. O-O { [%eval -0.36] } 9... c6 { [%eval -0.12] } 10. Re1 { [%eval -0.17] } 10... Bf5 { [%eval -0.04] } 11. c4?! { [%eval -0.67] } 11... dxc4 { [%eval -0.5] } 12. Bxc4 { [%eval -0.77] } 12... Nd7?! { [%eval -0.1] } 13. Nc3 { [%eval 0.0] } 13... Nb6 { [%eval 0.0] } 14. b3?! { [%eval -0.76] } 14... Nxc4 { [%eval -0.

ok so the positions dataset seems more directly easy to work with:
I have the board state in fen, the centipawn score, and the next move (line).
However, it doesn't have the previous move, if I wanted to use that as a feature.

the games dataset has the previous move as well, but is a little harder to work with:
- I'd need to filter down to games that have evaluations
- I'd need to reconstruct the game board by building up the moves.

I think for now I'm going to use the positions dataset, and I can always come back
and add the other feature later.